In [2]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.5/232.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0


In [3]:
import gc
import torch
import torch.optim as optim
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from SimpleBackwardAttention import SimpleBackwardAttention, CONFIG_DICT

In [4]:
device = torch.device('cpu')

In [5]:
dataset = load_dataset('nvidia/OpenMathInstruct-2')['train_1M']
data_list = []
for p, s, a in zip(dataset['problem'], dataset['generated_solution'], dataset['expected_answer']):
    prompt = f"prompt : {p}"
    answer = f"Reasoned_Solution: {s}\n So, the answer is {a}"
    data_list.append({'prompt': prompt, 'reasoned_solution': answer})
del dataset
data_list = data_list[:10]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.41k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

train-00000-of-00032.parquet:   0%|          | 0.00/237M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/13972791 [00:00<?, ? examples/s]

Generating train_1M split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating train_2M split:   0%|          | 0/2000000 [00:00<?, ? examples/s]

Generating train_5M split:   0%|          | 0/5000000 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/32 [00:00<?, ?it/s]

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
lm_head = GPT2LMHeadModel.from_pretrained('gpt2').to("cpu")
for param in lm_head.parameters():
    param.requires_grad = False
embeddings = lm_head.transformer.wte.weight.data.to("cpu")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
torch.manual_seed(123)
# Create the custom backward attention module and move it to CPU.
model = SimpleBackwardAttention(CONFIG_DICT).to(device)

In [9]:
def latch(token_ids: torch.Tensor, alpha=1.0, beta=1.0, eps=1e-8):
    """
    Given a tensor of token ids, obtain the logits from the frozen GPT-2 LM head (on CPU)
    and then use our backward attention module (on CPU) to produce a next-token probability distribution.
    """
    hyper_params = torch.tensor([alpha, beta], device=device, dtype=CONFIG_DICT['dtype'])
    hyper_params = F.softmax(hyper_params, dim=-1).to(device)

    # Run LM head on CPU.
    token_ids_cpu = token_ids.to("cpu")
    with torch.no_grad():
        outputs = lm_head(token_ids_cpu)
        logits_lm = outputs.logits[:, -1, :]  # shape: (batch, vocab_size)
    # Move logits to CPU (although they already are).
    logits_lm = logits_lm.to(device)

    lm_probs = F.softmax(logits_lm, dim=-1) * hyper_params[0]

    # Run backward attention without AMP (CPU uses full precision).
    model_logits = model(embeddings.to(device), logits_lm) * hyper_params[1]
    mixed_logits = lm_probs + model_logits + eps
    probs = F.softmax(mixed_logits, dim=-1)
    return probs

In [10]:
def supervised_loss(input_text: str, solution_text: str):
    """
    Compute the teacher-forcing cross-entropy loss for one example.
    The model is provided the prompt as context and then the target tokens are predicted one by one.
    """
    input_ids = tokenizer(input_text, return_tensors='pt', truncation=False)['input_ids'].to(device)
    solution_ids = tokenizer(solution_text, return_tensors='pt', truncation=False)['input_ids'].to(device)

    loss_total = torch.tensor(0.0, device=device)
    context = input_ids  # initial context
    seq_len = solution_ids.size(1)

    for i in range(seq_len):
        probs = latch(context)
        target_token = solution_ids[0, i]
        loss = -torch.log(probs[0, target_token] + 1e-8)
        loss_total += loss
        # Append target token to the context.
        target_token_tensor = target_token.unsqueeze(0).unsqueeze(0)
        context = torch.cat([context, target_token_tensor], dim=1)
    return loss_total / seq_len

In [11]:
train_dataloader = DataLoader(data_list, batch_size=2, shuffle=True)

In [12]:
def train():
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    num_epochs = 1
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0.0
        batch_count = 0
        for batch in train_dataloader:
            optimizer.zero_grad()
            batch_loss = torch.tensor(0.0, device=device)
            for prompt, solution in zip(batch['prompt'], batch['reasoned_solution']):
                loss = supervised_loss(prompt, solution)
                batch_loss += loss
                gc.collect()
            batch_loss = batch_loss / len(batch['prompt'])
            batch_loss.backward()
            optimizer.step()
            total_loss += batch_loss.item()
            batch_count += 1
            print(f"[Epoch {epoch + 1}] Batch {batch_count} - Loss: {batch_loss.item():.4f}")
            gc.collect()
        avg_loss = total_loss / batch_count if batch_count > 0 else 0.0
        print(f"Epoch {epoch + 1} completed. Average Loss: {avg_loss:.4f}")
        gc.collect()

In [ ]:
train()

/content/SimpleBackwardAttention.py:58: UserWarning: Expected the tensor to be same as torch.bfloat16but got torch.float32
 Trying to cast it to same dtype
  warnings.warn(f'Expected the tensor to be same as {dtype}'


[Epoch 1] Batch 1 - Loss: 10.5981


In [ ]:
torch.save(model.state_dict(), "simple_backward_attention_weights_cpu.pth")